In [1]:
import streamlit as st
import pandas as pd
import base64
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from PIL import Image
import pickle
import sklearn
import os

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [7]:
model1 = pickle.load(open('MLP_model.sav','rb'))
model2 = pickle.load(open('RF_model.sav','rb'))
model3 = pickle.load(open('GB_model.sav','rb'))

In [ ]:
path = ("./resources/Team_df.csv")
com_data = pd.read_csv(path)

In [4]:
# get index of every team's data
team_index = com_data['Team']

# Remove Opponent, Score, Result
mlp_model_data = com_data[['Team', 'Opp', 'TmScore', 'O_1stD', 'O_Tot_yd', 'O_P_Yd', 'O_R_Yd', 'O_TO',
                         'D_1stD', 'D_Tot_Yd', 'D_P_Yd', 'D_R_Yd', 'D_TO', 'Home','Prediction_LR','Prediction_ADA']]


# change to season stats
season_stats = ['O_1stD', 'O_Tot_yd', 'O_P_Yd', 'O_R_Yd', 'O_TO',
                         'D_1stD', 'D_Tot_Yd', 'D_P_Yd', 'D_R_Yd', 'D_TO']

mlp_model_data[season_stats] = mlp_model_data[season_stats] * 16

# standardise the data
from sklearn import preprocessing

sd_data = ['O_1stD', 'O_Tot_yd', 'O_P_Yd', 'O_R_Yd', 'O_TO',
                         'D_1stD', 'D_Tot_Yd', 'D_P_Yd', 'D_R_Yd', 'D_TO','Prediction_LR','Prediction_ADA']


mlp_model_data[sd_data] = preprocessing.scale(mlp_model_data[sd_data])

#get indexs of every teams
team_index = com_data['Team']

mlp_model_data = pd.get_dummies(mlp_model_data)

/opt/anaconda3/envs/dev/lib/python3.7/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/opt/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/dev/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [5]:
pd.options.display.max_rows = None

importance = GB_model.feature_importances_

feature_names = gb_model_data.drop('TmScore', axis=1).columns

d = {'Var_Name': feature_names, 'Imp': importance}
df = pd.DataFrame(data=d)
df = df.sort_values(by= ['Imp'], ascending = False).reset_index(drop=True)
df.head(15)

# function that return the score of two teams playing

pd.options.display.max_columns = None

# Create playoff test dataset from season averages

def Score_Predictor(home_team, away_team):
    team1 = home_team
    team2 = away_team
    
    team1_data = gb_model_data[com_data['Team'] == team1].drop('TmScore', axis=1).reset_index(drop=True)
    team2_data = gb_model_data[com_data['Team'] == team2].drop('TmScore', axis=1).reset_index(drop=True)
    
    week_slice = slice(1,16)
    
    #1 Remove if no team names
    team1_test = pd.DataFrame(team1_data[week_slice].mean(axis=0)).T #select week to use as average
    team1_test
    opp_columns = team1_test.filter(like='Opp').columns
    
    team1_test[opp_columns] = 0
    team1_test['Opp_' + team2] = 1
    team1_test['Home'] = 1
    
    #2
    team2_test = pd.DataFrame(team2_data[week_slice].mean(axis=0)).T #select week to use as average
    opp_columns = team2_test.filter(like='Opp').columns
    
    team2_test[opp_columns] = 0
    team2_test['Opp_' + team1] = 1
    team2_test['Home'] = 0 #remove home field advantage
    
    # head to head matchup
    team1_test[['D_1stD','D_Tot_Yd','D_P_Yd','D_R_Yd','D_TO']] = team2_test[['O_1stD','O_Tot_yd','O_P_Yd','O_R_Yd','O_TO']]
    team2_test[['D_1stD','D_Tot_Yd','D_P_Yd','D_R_Yd','D_TO']] = team1_test[['O_1stD','O_Tot_yd','O_P_Yd','O_R_Yd','O_TO']]
    
    X_Playoff_test = pd.concat([team1_test, team2_test])
    X_Playoff_test.fillna(0, inplace = True) 
    
    scores = GB_model.predict(X_Playoff_test)
    print(team1, "will score", round(scores[0], 1))
    print(team2, "will score", round(scores[1], 1))
    
    if scores[0] > scores[1]:
        winner = team1
    else:
        winner = team2
        
    print(winner, "are the WINNERS!!!")
    
    return scores, winner

NameError: name 'GB_model' is not defined

In [6]:
scores, winner = Score_Predictor('Philadelphia Eagles', 'Tampa Bay Buccaneers')

NameError: name 'Score_Predictor' is not defined